# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [2]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target, columns=['target'])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [5]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

404 102 404 102


Fit the model and apply the model to the make test set predictions

In [7]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

Calculate the residuals and the mean squared error

In [8]:
from sklearn.metrics import mean_squared_error

train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test

train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)

print('Train RMSE:', train_mse)
print('Test RMSE:', test_mse)

Train RMSE: 18.670762360906455
Test RMSE: 34.545590472134286


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [15]:
def kfolds(data, k):
    
    # Force data as pandas dataframe
    data = pd.DataFrame(data)
    rows = len(data)
    fold_len = rows // k
    num_extra = rows % k
    folds = []
    row_start = 0
    # Loop through fold splits
    for f in range(1, k+1):
        #Fold Size will be 1 larger to account for leftovers
        if f <= num_extra:            
            fold = data.iloc[row_start : row_start+fold_len+1] 
            folds.append(fold)
            row_start += fold_len + 1
        # No leftovers
        else:
            fold = data.iloc[row_start : row_start+fold_len] 
            folds.append(fold)
            row_start +=  fold_len
            
    return folds 

### Apply it to the Boston Housing Data

In [16]:
# Make sure to concatenate the data again
boston_data = pd.concat([X.reset_index(drop=True), y], axis=1)

In [17]:
boston_kfolds = kfolds(boston_data, 5)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [19]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(boston_kfolds) if i!=n])
    test = boston_kfolds[n]
    
    # Fit a linear regression model
    linreg.fit(train[X.columns], train[y.columns])
    
    #Evaluate Train and Test Errors
    y_hat_train = linreg.predict(train[X.columns])
    y_hat_test = linreg.predict(test[X.columns])
    train_residuals = y_hat_train - train[y.columns]
    test_residuals = y_hat_test - test[y.columns]
    train_errs.append(np.mean(train_residuals.astype(float)**2))
    test_errs.append(np.mean(test_residuals.astype(float)**2))
print('Train Errors:', train_errs)
print('\nTest Errors:', test_errs)

Train Errors: [target    24.195577
dtype: float64, target    23.032087
dtype: float64, target    19.745073
dtype: float64, target    15.317101
dtype: float64, target    22.329973
dtype: float64]

Test Errors: [target    13.405145
dtype: float64, target    17.444017
dtype: float64, target    37.032711
dtype: float64, target    58.279544
dtype: float64, target    26.097989
dtype: float64]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [26]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

boston_cv5 = cross_val_score(linreg, X, y, cv=5, scoring="neg_mean_squared_error")
boston_cv5

array([-13.40514492, -17.4440168 , -37.03271139, -58.27954385,
       -26.09798876])

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [28]:
print('Mean MSE over 5 Cross-Validations:', np.mean(boston_cv5))
# The result for the train-test-split case (test_size = 0.2) above is 34.55. 
# We see that the cross_val_score() function results are equal in magnitude but opposite in sign from the 
# kfolds() function results. 
# Ignoring the sign, we find that the average Scikit-Learn Cross-Validation results are a little better than
# the train-split-test case.

Mean MSE over 5 Cross-Validations: -30.451881143540323


##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!